In [9]:
lines = []
with open("Automated_Traffic_Volume_Counts_20241127.csv", "r") as f:
    lines = f.readlines()[1:]

In [10]:
pos2id = {}

for line in lines:
    xs = line.split(",")
    pos = xs[9]
    if not pos in pos2id:
        num = len(pos2id.keys())
        pos2id[pos] = num

In [11]:
len(pos2id)

3598

In [18]:
streets = []
for i in range(len(pos2id)):
    streets.append(set())
for line in lines:
    xs = line.split(",")
    for j in range(10, 13):
        if not "Dead end" in xs[j]:
            streets[pos2id[xs[9]]].add(xs[j])
    
    

In [ ]:
import pyproj

s_crs = pyproj.CRS.from_string("EPSG:2263")
t_crs = pyproj.CRS.from_string("EPSG:4326")

x, y = 997407.0998491726, 208620.92612708386

transformer = pyproj.Transformer.from_crs(s_crs, t_crs, always_xy=True)
t_p = transformer.transform(x, y)
print(t_p)

(-73.95252157233777, 40.7392830527809)


建图

In [31]:
id2poswkt = [(1.,1.)] * N
for k, v in pos2id.items():
    k = k[7:-1].split(" ")
    x, y = float(k[0]), float(k[1])
    id2poswkt[v] = (x, y)

In [32]:
import numpy as np

N = len(pos2id)
edges = set()
A = np.zeros((N, N), dtype=int)
road2ids = {}
for i, street in enumerate(streets):
    for s in street:
        if not s in road2ids:
            road2ids[s] = [i]
        else:
            road2ids[s].append(i)

for k, v in road2ids.items():
    n = len(v)
    lst = list(v)
    lst.sort(key=lambda e: id2poswkt[e][0])
    for i in range(n-1):
        A[lst[i], lst[i+1]] = A[lst[i+1], lst[i]] = 1

A.sum()
    

8334

In [27]:
id2pos = [(1.,1.)] * N
for k, v in pos2id.items():
    k = k[7:-1].split(" ")
    x, y = float(k[0]), float(k[1])
    yy, xx = transformer.transform(x, y)
    id2pos[v] = (yy, xx)

In [29]:
id2poswkt = [(1.,1.)] * N
for k, v in pos2id.items():
    k = k[7:-1].split(" ")
    x, y = float(k[0]), float(k[1])
    id2poswkt[v] = (x, y)

In [33]:
with open("NYTV_wkt.csv", "x") as f:
    f.write("from,to,cost\n")
    for i in range(N):
        for j in range(i+1, N):
            if A[i,j]:
                id1, id2 = i, j
                pos1, pos2 = id2poswkt[id1], id2poswkt[id2]
                cost = np.sqrt((pos1[0]-pos2[0])**2 + (pos1[1]-pos2[1])**2)
                f.write(",".join([str(id1), str(id2), str(cost)]) + "\n")

In [35]:
minn, maxx = 202512302345, 0
for line in lines:
    line = line.split(",")
    tmp = int(line[2]) * 100000000 + int(line[3]) * 1000000 + int(line[4]) * 10000 + int(line[5]) * 100 + int(line[6])
    minn = min(minn, tmp)
    maxx = max(maxx, tmp)
minn, maxx

(200001010015, 202406102345)

In [37]:
time2v = {}
for line in lines:
    line = line.split(",")
    tmp = int(line[2]) * 1000000 + int(line[3]) * 10000 + int(line[4]) * 100 + int(line[5])
    if not tmp in time2v:
        time2v[tmp] = np.zeros((N, 1), dtype=int)
    idd = pos2id[line[9]]
    time2v[tmp][idd,0] += int(line[7])

In [42]:
datas = []
timestp = list(time2v.keys())
timestp.sort()
for time_ in timestp:
    datas.append(time2v[time_].reshape(1, -1, 1))


In [44]:
np.vstack(datas).shape

(117085, 3598, 1)

In [45]:
np.savez("NYTV.npz", data=np.vstack(datas))

In [1]:
import numpy as np

a = np.load("NYTV1.npz")
a["data"].shape

(117085, 3598, 1)

In [2]:
b = a["data"][-20000:,:,:]
b.shape

(20000, 3598, 1)

In [3]:
np.savez("NYTV.npz", data=b)

In [1]:
import numpy as np

N = 3598
A = np.zeros((N, N), dtype=np.int8)
with open("NYTV_wkt.csv", "r") as f:
    lines = f.readlines()[1:]
    for line in lines:
        a = line.strip().split(",")
        i, j = int(a[0]), int(a[1])
        A[i,j]=A[j,i]=1



In [2]:
nb_nums = A.sum(axis=0)
topk_nodes = nb_nums.argsort()[-10:]
topk_nodes

array([1510, 2147, 1589, 2958, 1520,  888,  889, 2186,  527, 3358],
      dtype=int64)

In [3]:
nodes = set()
for nd in topk_nodes:
    nodes.add(nd)

while True:
    if len(nodes) >= 500:
        break
    lst = []
    for node in nodes:
        for i in range(N):
            if A[i,node]:
                lst.append(i)
    for i in lst:
        nodes.add(i)

len(nodes)

507

In [4]:
# 重新映射
new_N = len(nodes)
new_id = {}
num = 0
for node in nodes:
    new_id[node] = num
    num += 1

In [5]:
len(lines)

4167

In [14]:
with open("NYTV.csv", "x") as f:
    f.write("from,to,cost\n")
    for line in lines:
        a = line.strip().split(",")
        i, j = int(a[0]), int(a[1])
        if i in nodes and j in nodes:
            new_line = ",".join([str(new_id[i]), str(new_id[j]), a[2]]) + "\n"
            f.write(new_line)
            

In [22]:
import numpy as np

a = np.load("NYTV1.npz")
a["data"].shape

(117085, 3598, 1)

In [16]:
new_data = np.zeros((20000, new_N, 1), dtype=np.uint16)
old_id = [0] * new_N
for k, v in new_id.items():
    old_id[v] = k
for i in range(new_N):
    new_data[:,i,0] = a["data"][:,old_id[i],0]

In [8]:
old_id = {}
for k, v in new_id.items():
    old_id[v] = k

In [12]:
id2pos = [0] * N
for k, v in pos2id.items():
    id2pos[v] = k

In [23]:
t = a["data"][-12:, :, :]
t.shape

(12, 3598, 1)

In [24]:
x = np.random.uniform(0.5, 1.5, (12, 3598, 1))

In [25]:
tt = t * x

In [26]:
tt.mean()

0.7791202858993543

In [29]:
import pyproj

s_crs = pyproj.CRS.from_string("EPSG:2263")
t_crs = pyproj.CRS.from_string("EPSG:4326")

x, y = 997407.0998491726, 208620.92612708386

transformer = pyproj.Transformer.from_crs(s_crs, t_crs, always_xy=True)
t_p = transformer.transform(x, y)
print(t_p)

(-73.95252157233777, 40.7392830527809)


In [35]:
year = 2024
month = 6
day = 11
def id2jingwei(idd):
    pos = id2pos[idd]
    pos = pos[7:-1]
    pos = pos.split(" ")
    return transformer.transform(float(pos[0]), float(pos[1]))

with open("predict.csv", "x") as f:
    f.write("lat,lon,year,month,day,hour,total_volume\n")
    for i in range(3598):
        lon, lat = id2jingwei(i)
        for h in range(12):
            f.write("%f,%f,2024,06,11,%d,%f\n" % (lat, lon, h, float(tt[h,i,0])))

In [37]:
np.where(tt != 0)[0].sum()

132